In [1]:
%matplotlib qt5
%load_ext autoreload
%autoreload 2

import cv2
import numpy as np
import rospy, rosbag, tf
import os, csv, time, argparse, math
from matplotlib import pyplot as plt

from nav_msgs.msg import Odometry
from geometry_msgs.msg import Twist
from gazebo_msgs.msg import ModelStates
from sensor_msgs.msg import Image, CameraInfo
from cv_bridge import CvBridge, CvBridgeError
from tf.transformations import euler_from_quaternion, quaternion_from_euler

from src.VBOATS import VBOATS

In [2]:
class image_converter:
    def __init__(self):
        rospy.init_node('vboat_pipeline')
        self.bridge = CvBridge()
        self.image_pub = rospy.Publisher("/obstacles",Image,queue_size=1000)
        self.image_sub = rospy.Subscriber("/ugv1/d415/depth/image_raw",Image,self.callback)
        
        self.vboat = VBOATS()
        self.vboat.dead_x = 0
        self.vboat.dead_y = 6
        self.r = rospy.Rate(40)
        self.img = []
        self.obs_disp = []
        self.flag_initialization = False
        self.nSamples = 100
        self.dSum = 0
        self.dmaxAvg = 0
        self.count = 0
        self.disp_obs = None
        
    def callback(self,data):
        try:
            cv_image = self.bridge.imgmsg_to_cv2(data, "16UC1")
            cv_image = np.float32(cv_image)
        except CvBridgeError as e: print(e)
                
        if(not self.flag_initialization):
            tmp = cv_image/65535
            depth = np.uint8(tmp*255)
#             depth = cv2.cvtColor(depth,cv2.COLOR_GRAY2BGR)
            self.img = np.copy(depth)
            self.vboat.pipeline(depth, threshU1=11,threshU2=20, threshV2=70)
            display_obstacles = cv2.cvtColor(self.vboat.img, cv2.COLOR_GRAY2BGR)

            for ob in self.vboat.obstacles:
                cv2.rectangle(display_obstacles,ob[0],ob[1],(150,0,0),1)

            self.disp_obs = np.copy(display_obstacles)
            try:
                self.obs_disp = np.copy(display_obstacles)
                self.image_pub.publish(self.bridge.cv2_to_imgmsg(display_obstacles, "bgr8"))
            except CvBridgeError as e:
                print(e)
            
    def start(self):
        while not rospy.is_shutdown():
            self.r.sleep()

In [3]:
io = image_converter()

In [15]:
plt.figure(1)
plt.imshow(io.vboat.umap_raw)
plt.show()

plt.figure(2)
plt.imshow(io.vboat.umap_processed)
plt.show()

plt.figure(3)
plt.imshow(io.vboat.vmap_raw)
plt.show()

print(io.vboat.dmax)

256


In [4]:
# D415 Parameters returned from real camera
rotation = [1, 0.000451149, 0.000541019, -0.000453043, 0.999994, 0.0035053, -0.000539434, -0.00350554, 0.999994]
translation = [0.0150125, 2.66028e-05, -0.000210271]
intrinsics = {
    "width": 640, "height": 480, 
    "ppx": 320.551, "ppy": 232.202, 
    "fx": 626.464, "fy": 626.464, 
    "model": "Brown Conrady", 
    "coeffs": [0, 0, 0, 0, 0]
}

F = np.array([
    [intrinsics['fx'], 0, intrinsics['ppx']],
    [0, intrinsics['fy'], intrinsics['ppy']],
    [0,0,1]
])
R = np.array(rotation).reshape((3, 3))
Rinv = np.linalg.inv(R)
T = np.array(translation).reshape((3, 1))
T2 = np.array([intrinsics['ppx']*0.001, intrinsics['ppy']*0.001, 0, 0]).T
print T.T
print T2.T

[[ 1.50125e-02  2.66028e-05 -2.10271e-04]]
[0.320551 0.232202 0.       0.      ]


In [5]:
def get_camera_pose(cam_frame = '/ugv1/d415_camera_depth_optical_frame',base_frame = '/ugv1/odom'):
    listener = tf.TransformListener()
    listener.waitForTransform(base_frame,cam_frame, rospy.Time(0), rospy.Duration(8.0))
    (trans,rot) = listener.lookupTransform(base_frame,cam_frame, rospy.Time(0))
    roll,pitch,yaw = tf.transformations.euler_from_quaternion(rot)

    pose = np.array(trans+[np.rad2deg(yaw)])
    Tmat = np.array(trans).T
    Rmat = tf.transformations.quaternion_matrix(rot)
    return pose, Rmat, Tmat

def get_distance(umap, xs, ds, ys, focal=[462.138,462.138], baseline=0.055, dscale=0.001, pp=[320.551,232.202], method=0):
        nonzero = umap.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])

        good_inds = ((nonzeroy >= ds[0]) & (nonzeroy < ds[1]) &
                     (nonzerox >= xs[0]) &  (nonzerox < xs[1])).nonzero()[0]

        xmean = np.int(np.mean(nonzerox[good_inds]))
        ymean = np.mean(ys)
        dmean = np.mean(nonzeroy[good_inds])
        
        if(method == 1): 
            dist = (focal[0]*baseline)/dmean
            z = dist*dscale
        else:
#             z = np.min(ds)*(65535/255)*dscale
            z = dmean*(65535/255)*dscale
        
        # float x = (pixel[0] - intrin->ppx) / intrin->fx;
        # float y = (pixel[1] - intrin->ppy) / intrin->fy;
#         x = (xmean - pp[0])/focal[0]
#         y = (ymean - pp[1])/focal[1]
        
        x = (xmean)/focal[0]
        y = (ymean)/focal[1]
        
        x = x * z
        y = y * z
        print("Distance, X, Y: %.3f, %.3f, %.3f" % (z, x,y))
        return z, x,y, dmean

In [20]:
test_terra = False
disp = np.zeros_like(io.vboat.img)
disp = cv2.cvtColor(disp,cv2.COLOR_GRAY2BGR)

umap = io.vboat.umap_raw
xs = io.vboat.xBounds
ds = np.array(io.vboat.dbounds)
obs = io.vboat.obstacles

ppx = 320.551; ppy = 232.202; fx = 626.464; fy = 626.464; b = 0.055
pp = [ppx, ppy];   focal = [fx,fy]
dGain = (65535/255)

if(test_terra):
    obid = i = 0
    # Obstacles terra world
    X0 = np.array([
        [4.0, -1.0, 0.227],
        [3.0, 0, 0.177],
        [2.0, -2.0, 0.344]
    ])
else:
    obid = 2
    i = 0
    # Obstacles offset world
    X0 = np.array([
        [11.59, -3.6, 0.21336],
        [8.576193, -1.165787, 0.21336],
        [5.694473, 2.2, 0.21336]
    ])


pose,RotM,Tmat = get_camera_pose()
Xk = pose[:2]; yaw = pose[-1]; ang = np.deg2rad(yaw)
x1 = X0[obid,0];     y1 = X0[obid,1]
dx = x1 - Xk[0];     dy = y1 - Xk[1]
true_dist = np.sqrt(dx*dx + dy*dy)-X0[obid,2]

print "Obstacle Disparities:\r\n", ds
print
d = np.mean(ds[i,0])
z = d * dGain * 0.001
tmpD = d* dGain
dist = (fx*0.0150125)/(tmpD*0.001)
# print(tmpD, z, dist)

# Method 2: --------
disparities = ds[i]
us = [obs[i][0][0], obs[i][1][0]]
vs = [obs[i][0][1], obs[i][1][1]]
z2,ux,uy,_ = get_distance(umap,us,disparities,vs, focal=focal,pp=pp)

# print "Obstacle X pixel ranges: ", us, "\r\nObstacle Y pixel ranges: ", vs
print("True Dist, Est. Dist (Method #1), (Method #2): %.3f, %.3f, %.3f" % (true_dist, z, z2) )

pxl = np.array([ [ux],[uy],[z] ])
tpxl = np.vstack([pxl,1])

RotMinv = np.linalg.inv(RotM)
pos = np.dot(RotMinv, tpxl)-np.vstack([T,0])
tpos = np.vstack([pos,1])

tmp1 = np.dot(RotM,tpxl)-Tmat
# tmp2 = np.dot(mat,tpos)

print
print "Pixel (X,Y,Z): ", pxl.T[0]
print "Position (X,Y,Z): ", pos.T[0]
print "Temp (X,Y,Z): ", tmp1.T[0]
print
# print np.dot(RotM,pos[:2]).T
print

xoff = np.tan(ang)*z
yoff = z / np.cos(ang)
zoff = (fx*0.001)/np.tan(ang)

offs = [xoff, yoff, zoff]

print
print "Offsets: ", offs
print

# location = [yoff, np.asscalar(xoff-pos[0])]
location = [yoff, np.asscalar(xoff+pos[0])]
# print pos
print "Camera Location: ", Xk, yaw
print "Obstacle Estimated Location: ", location
print "Obstacle True Location: ", X0[obid,:2]

Obstacle Disparities:
[[ 9 11]]

Distance, X, Y: 2.420, 1.298, 1.004
True Dist, Est. Dist (Method #1), (Method #2): 3.498, 2.313, 2.420

Pixel (X,Y,Z):  [1.29777964 1.00423425 2.313     ]
Position (X,Y,Z):  [ 1.61635077 -2.31262357 -0.18209875  1.        ]
Temp (X,Y,Z):  [-9.28220379 -6.04064198 -9.6091817  -7.60486947]



Offsets:  [2.2413414529787055, 3.220804326381951, 0.6464928536766624]

Camera Location:  [ 8.60486947 -0.10303901] 44.098575810654054
Obstacle Estimated Location:  [3.220804326381951, 3.857692222454809]
Obstacle True Location:  [5.694473 2.2     ]


In [ ]:
34212.933/3600


In [ ]:
bot = [0.444806, -0.559817]
obs = [2.000000, -2]
dx = bot[0] - obs[0]
dy = bot[1] - obs[1]
dist = np.sqrt(dx*dx + dy*dy)
print(dist)

In [ ]:
win = io.vboat.obstacles_umap[0]
print(win)
crop_img = io.vboat.umap_raw[win[0][1]:win[1][1], win[0][0]:win[1][0]]

tmpMean = np.mean(crop_img)
print(tmpMean)

plt.figure(3)
plt.imshow(crop_img)
plt.show()

In [ ]:
# cam_frame = '/ugv/d415_camera_depth_optical_frame'
cam_frame = '/ugv1/d415_camera_depth_optical_frame'
base_frame = '/ugv1/odom'
listener = tf.TransformListener()
listener.waitForTransform(base_frame,cam_frame, rospy.Time(0), rospy.Duration(8.0))
(trans,rot) = listener.lookupTransform(base_frame,cam_frame, rospy.Time(0))
# listener.asMatrix(cam_frame)
mat = tf.transformations.quaternion_matrix(rot)
print(mat)

In [ ]:
tmp = np.dot(mat,np.vstack([pos,1]))
print tmp

In [ ]:
tmp = np.dot(mat,np.vstack([pos,1]))
print tmp
print
print pos
print
print xoff,yoff,zoff

In [ ]:
math.atan((0.5*640)/626.464)/0.5